In [1]:
import os
import json
import csv

## Json to CSV files/joining

In [9]:
data_folder_videos='/media/aruiz/data/videos_data/'
filename1='general_videos_data2.json'
filename2='general_videos_data.csv'
file1=os.path.join(data_folder_videos,filename1)
file2=os.path.join(data_folder_videos,filename2)
file3= "json_errors.txt"

fieldnames = ['id', 'publishedAt', 'title','description','tags','categoryId','liveBroadcast','channelId','channelTitle',
              'favoriteCount','commentCount','viewCount','likeCount','dislikeCount','duration','definition', 'caption',
              'topicIds', 'freebaseTopIds']
output=open(file2, 'a')

writer = csv.DictWriter(output, fieldnames=fieldnames)
# writer.writeheader()
i=0

with open (file1,'r') as f:
    for line in f:
        i+=1
        if i>0:
            try:
                clean_dict={}
                json_dict=json.loads(line)
                videos= json_dict.get("items",[])
                if (videos!=[]):
                    clean_dict['id']= videos[0]['id']
                    clean_dict['publishedAt']= videos[0]['snippet']['publishedAt']
                    clean_dict['channelId']= videos[0]['snippet']['channelId']
                    clean_dict['title']= videos[0]['snippet']['title']
                    
                    if 'channelTitle' in videos[0]['snippet']:
                        clean_dict['channelTitle']= videos[0]['snippet']['channelTitle']
                    
                    if 'viewCount' in videos[0]['statistics']:
                        clean_dict['viewCount']=  videos[0]['statistics']['viewCount']

                    if 'description' in videos[0]['snippet']:
                        clean_dict['description']= videos[0]['snippet']['description']

                    if 'tags' in videos[0]['snippet']:
                        clean_dict['tags']= videos[0]['snippet']['tags']

                    if 'categoryId' in videos[0]['snippet']:
                        clean_dict['categoryId']= videos[0]['snippet']['categoryId']

                    if 'liveBroadcastContent' in videos[0]['snippet']:
                        clean_dict['liveBroadcast']= videos[0]['snippet']['liveBroadcastContent']

                    if 'favoriteCount' in videos[0]['statistics']:
                        clean_dict['favoriteCount']= videos[0]['statistics']['favoriteCount']

                    if 'commentCount' in videos[0]['statistics']:
                        clean_dict['commentCount']= videos[0]['statistics']['commentCount']

                    if 'duration' in videos[0]['contentDetails']:
                        clean_dict['duration']= videos[0]['contentDetails']['duration']

                    if 'definition' in videos[0]['contentDetails']:
                        clean_dict['definition']= videos[0]['contentDetails']['definition']

                    if 'caption' in videos[0]['contentDetails']:
                        clean_dict['caption']= videos[0]['contentDetails']['caption']

                    if 'topicDetails' in videos[0]:
                        if 'topicIds' in videos[0]['topicDetails']:
                            clean_dict['topicIds']= videos[0]['topicDetails']['topicIds']
                        if 'relevantTopicIds' in videos[0]['topicDetails']:
                            clean_dict['freebaseTopIds']= videos[0]['topicDetails']['relevantTopicIds']

                    if 'likeCount' in videos[0]['statistics']:
                        clean_dict['likeCount']= videos[0]['statistics']['likeCount']

                    if 'dislikeCount' in videos[0]['statistics']:
                        clean_dict['dislikeCount']= videos[0]['statistics']['dislikeCount']

                writer.writerow(clean_dict)
                
            except json.JSONDecodeError:
                errors=open(file3, 'a')
                errors.write(str(i)+'\n')
                errors.close()
                continue

In [2]:
#total=9632501
#total 2 = 4365722
#total general = 13998223
#no duplicates = 11752812

### Counting lines in csv

In [29]:
with open (file4, 'r') as csvfile:
    fileObj=csv.reader(csvfile)
    num = sum(1 for row in fileObj)
num

11752812

## Clean duplicates

In [28]:
import os
import csv
import pandas as pd

total_g = 13998222
fieldnames = ['id', 'publishedAt', 'title','description','tags','categoryId','liveBroadcast','channelId','channelTitle',
              'favoriteCount','commentCount','viewCount','likeCount','dislikeCount','duration','definition', 'caption',
              'topicIds', 'freebaseTopIds']
data_folder_videos='/media/aruiz/data/videos_data/'
filename2='general_videos_data.csv'
file4=os.path.join(data_folder_videos,'general_videos_clean.csv')
file2=os.path.join(data_folder_videos,filename2)
# df=dd.read_csv(file2,sep =',', header=0,encoding='utf-16')
# df.head()

# --------------- divide csv in chunks ---------------------------------------------------
df = pd.read_csv(file2, header=0, names=fieldnames, encoding='utf-8', chunksize=1000000)
ids=set()
j=0

for chunk in df:
    j+=1
    chunk = chunk.drop_duplicates(['id'], keep='first')
    chunk = chunk[~chunk['id'].isin(ids)]
    ids.update(chunk['id'].values)
    chunk.to_csv(file4, encoding='utf-8', index=False, chunksize=1000000, mode='a')

/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Save each community videos in a csv

In [27]:
data_folder_channels='/media/aruiz/data/videos_data/'
file1=os.path.join(data_folder_channels,'general_videos_clean.csv')
file2=os.path.join(data_folder_channels,'node_data_bc.csv')
fieldnames = ['id', 'publishedAt', 'title','description','tags','categoryId','liveBroadcast','channelId','channelTitle',
              'favoriteCount','commentCount','viewCount','likeCount','dislikeCount','duration','definition', 'caption',
              'topicIds', 'freebaseTopIds']

#community chosen
community_num=[9,22,85,76,71,37,87]

for ind in range (1,len(community_num)):

    # create csv to save community videos
    file3=os.path.join(data_folder_channels,'data_community_'+str(community_num[ind])+'.csv')

    # Data frame node data from graph communities
    df1=pd.read_csv(file2)
    channel_com = df1.drop(df1[df1['modularity_class']!= community_num[ind]].index)
    com_ids=set()
    com_ids.update(channel_com['Id'].values)

    # Data frame all videos 
    df2 = pd.read_csv(file1, header=0, names=fieldnames, encoding='utf-8', chunksize=1000000)

    for chunk in df2:
        chunk = chunk[chunk['channelId'].isin(com_ids)]
        chunk.to_csv(file3, encoding='utf-8', index=False, chunksize=1000000, mode='a')